* Notebook created by nov05 on 2024-12-29
* It was run locally with conda env `sagemaker_py310`.  

---   
* View the S3 bucket in your account   
    https://s3.console.aws.amazon.com/s3/buckets/aft-vbi-pds
* [Docs > Models and pre-trained weights > ResNet > resnet34](https://pytorch.org/vision/main/models/generated/torchvision.models.resnet34.html)  
* GitHub gist [code snippets](https://gist.github.com/nov05/95cb7edcbe2e8bb68c9d29bdc00b9ca8)   

* Check [SageMaker AI Pricing](https://aws.amazon.com/sagemaker-ai/pricing/) > On-Demand Pricing > Training  
    | Instance Type      | vCPU | Memory  | Price per Hour |
    |--------------------|------|---------|----------------|
    | ml.g4dn.xlarge      | 4    | 16 GiB  | $0.736         |
    |ml.p3.2xlarge	| 8	| 61 GiB	| $3.825 | 

* Documentation > Amazon SageMaker > Developer Guide   
  [**Use the PyTorch framework estimators in the SageMaker Python SDK**](https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel-framework-estimator.html)    

* sagemaker 2.239.0  
  [**PyTorch Guide to SageMaker’s distributed data parallel library**](https://sagemaker.readthedocs.io/en/stable/api/training/sdp_versions/v1.0.0/smd_data_parallel_pytorch.html)  

In [2]:
%pwd

'd:\\github\\udacity-nd009t-capstone-starter\\starter'

In [5]:
## windows cmd to launch notepad to edit aws credential file
# !notepad C:\Users\guido\.aws\config
!notepad C:\Users\guido\.aws\credentials

In [6]:
## reset the session after updating credentials
import boto3 # type: ignore
boto3.DEFAULT_SESSION = None
import sagemaker # type: ignore
from sagemaker import get_execution_role # type: ignore

# Extract and print the account ID
sts_client = boto3.client('sts')
response = sts_client.get_caller_identity() 
account_id = response['Account']

role_arn = get_execution_role()  ## get role ARN
if 'AmazonSageMaker-ExecutionRole' not in role_arn:
    ## Go to "IAM - Roles", search for "SageMaker", find the execution role.
    voclabs_role_arn = role_arn
    sagemaker_role_arn = "arn:aws:iam::570668189909:role/service-role/AmazonSageMaker-ExecutionRole-20250126T194519"
session = sagemaker.Session()  ## "default"
region = session.boto_region_name
bucket = session.default_bucket()

print(f"Current AWS Account ID: {account_id}")
print("AWS Region: {}".format(region))
print("Default Bucket: {}".format(bucket))
print(f"Role voclabs ARN: {voclabs_role_arn}") ## If local, Role ARN: arn:aws:iam::807711953667:role/voclabs
print("SageMaker Role ARN: {}".format(sagemaker_role_arn)) 

## generate secrets.env. remember to add it to .gitignore  
import wandb
wandb.sagemaker_auth(path="../secrets") 

## get my own AWS account info
def get_secrets(name):
    path = '../secrets/' + name
    with open(path, 'r') as file:
        for line in file:
            return line.strip()
aws_account_number = get_secrets('aws_account_number')
aws_account_profile = get_secrets('aws_account_profile')

d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\pydantic\_internal\_fields.py:192: UserWarning: Field name "json" in "MonitoringDatasetFormat" shadows an attribute in parent "Base"
  warnings.warn(


[02/08/25 03:35:51] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=831461;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=552856;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

sagemaker.config INFO - Not applying SDK defaults from location: C:\ProgramData\sagemaker\sagemaker\config.yaml
sagemaker.config INFO - Not applying SDK defaults from location: C:\Users\guido\AppData\Local\sagemaker\sagemaker\config.yaml


[02/08/25 03:35:52] INFO     Found credentials in shared credentials file: ~/.aws/credentials   ]8;id=582718;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py\credentials.py]8;;\:]8;id=405955;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\botocore\credentials.py#1278\1278]8;;\

[02/08/25 03:35:56] WARNING  Couldn't call 'get_role' to get Role ARN from role name voclabs to get ]8;id=202352;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=565470;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#5971\5971]8;;\
                             Role path.                                                                            

Current AWS Account ID: 570668189909
AWS Region: us-east-1
Default Bucket: sagemaker-us-east-1-570668189909
Role voclabs ARN: arn:aws:iam::570668189909:role/voclabs
SageMaker Role ARN: arn:aws:iam::570668189909:role/service-role/AmazonSageMaker-ExecutionRole-20250126T194519


## 👉 **Data Preparation** (Run only once)
**TODO:** Run the cell below to download the data.

The cell below creates a folder called `data`, downloads training data and arranges it in subfolders. Each of these subfolders contain images where the number of objects is equal to the name of the folder. For instance, all images in folder `1` has images with 1 object in them. Images are not divided into training, testing or validation sets. If you feel like the number of samples are not enough, you can always download more data (instructions for that can be found [here](https://registry.opendata.aws/amazon-bin-imagery/)). However, we are not assessing you on the accuracy of your final trained model, but how you create your machine learning engineering pipeline.

* View the S3 bucket in your account   
    https://s3.console.aws.amazon.com/s3/buckets/aft-vbi-pds
    

In [ ]:
import os
import json
from tqdm import tqdm
import boto3
from botocore import UNSIGNED
from botocore.client import Config

def download_and_arrange_data(
        prefix='bin-images', 
        file_extension='.jpg',
        download_dir='../data/bin-images',
        partition=True):
    
    s3_client = boto3.client('s3', config=Config(signature_version=UNSIGNED))  ## public access

    ## There are 140536 image file names in the list. 
    with open('file_list.json', 'r') as f:
        d = json.load(f)

    for k, v in d.items():  ## There are 5 items (for 5 classes) in the JSON file.
        print(f"Downloading images/metadata of images with {k} object...")
        if partition:
            download_dir = os.path.join(download_dir, k)
        if not os.path.exists(download_dir):
            os.makedirs(download_dir)
        for file_path in tqdm(v):
            file_name = os.path.basename(file_path).split('.')[0] + file_extension
            s3_client.download_file(
                'aft-vbi-pds', 
                prefix+'/'+file_name,  ## e.g. metadata/100313.json
                download_dir+'/'+file_name)
            
## download the 10K-dataset metadata, 17.9 MB, 56m 57.4s
download_and_arrange_data(
    prefix='metadata', 
    file_extension='.json',
    download_dir='../data/metadata',
    partition=False)
print("total metadata file number:", 1228 + 2299 + 2666 + 2373 + 1875)

```text
Downloading images/metadata of images with 1 object...
100%|██████████| 1228/1228 [06:36<00:00,  3.09it/s]
Downloading images/metadata of images with 2 object...
100%|██████████| 2299/2299 [12:38<00:00,  3.03it/s]
Downloading images/metadata of images with 3 object...
100%|██████████| 2666/2666 [14:35<00:00,  3.04it/s]
Downloading images/metadata of images with 4 object...
100%|██████████| 2373/2373 [12:54<00:00,  3.06it/s]
Downloading images/metadata of images with 5 object...
100%|██████████| 1875/1875 [10:11<00:00,  3.07it/s]

total metadata file number: 10441
```

## 👉 **Convert 10K-dataset on S3 to WebDataset tar files with SageMaker ScriptProcessor on a custome image** (Run only once) 

In [ ]:
%%time
## TODO: Perform any data cleaning or data preprocessing
## This cell shuffle then split the 10K dataset to train, val, and test.  
## And convert the datasets to WebDataset tar files for SageMaker FastFile input mode.
from sagemaker.processing import ScriptProcessor
processor = ScriptProcessor(
    command=['python3'],
    ## You can use a custom image or use the default SageMaker image
    ## You can pull from AWS ECR or DockerHub
    image_uri=f'{aws_account_number}.dkr.ecr.us-east-1.amazonaws.com/udacity/p5-amazon-bin-images:latest', 
    role=sagemaker_role_arn,  # Execution role
    instance_count=1,
    instance_type='ml.t3.large',  # Use the appropriate instance type
    volume_size_in_gb=10,  # Minimal disk space since we're streaming
    base_job_name='p5-amazon-bin-images' 
)
processor.run(
    code='../scripts_process/convert_to_webdataset_10k.py',  # process the 10K files in the list
    arguments=[
        '--SM_INPUT_BUCKET', 'aft-vbi-pds',
        '--SM_INPUT_PREFIX_IMAGES', 'bin-images/',
        '--SM_INPUT_PREFIX_METADATA', 'metadata/',
        '--SM_OUTPUT_BUCKET', 'p5-amazon-bin-images',
        '--SM_OUTPUT_PREFIX', 'webdataset/',
    ]
)
## It took about 13 minutes to process 10.4K files (1.2 GB). If we keep 1K files per shard, 
## processing 500K files could take around 11 hours. I’ll probably increase it to 10K 
## files per shard, which would make each tar file around 1 GB and speed up the process.
## CPU times: total: 21.9 s
## Wall time: 12min 58s

[01/31/25 17:55:50] INFO     Creating processing-job with name                                      ]8;id=28680;file://d:\Users\guido\miniconda3\envs\sagemaker_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=800486;file://d:\Users\guido\miniconda3\envs\sagemaker_py310\lib\site-packages\sagemaker\session.py#1575\1575]8;;\
                             p5-amazon-bin-images-2025-01-31-23-55-46-542                                          

........................................................................Starting data processing...
🟢 File list successfully loaded from s3://p5-amazon-bin-images/file_list.json
    Total number of image files: 10441
# writing train-shard-000000.tar 0 0.0 GB 0
# writing train-shard-000001.tar 1000 0.1 GB 1000
# writing train-shard-000002.tar 1000 0.1 GB 2000
# writing train-shard-000003.tar 1000 0.1 GB 3000
# writing train-shard-000004.tar 1000 0.1 GB 4000
# writing train-shard-000005.tar 1000 0.1 GB 5000
# writing train-shard-000006.tar 1000 0.1 GB 6000
# writing train-shard-000007.tar 1000 0.1 GB 7000
🟢 Successfully uploaded shard files to s3://p5-amazon-bin-images/webdataset/train/:
    ['train-shard-000000.tar', 'train-shard-000001.tar', 'train-shard-000002.tar', 'train-shard-000003.tar', 'train-shard-000004.tar', 'train-shard-000005.tar', 'train-shard-000006.tar', 'train-shard-000007.tar']
# writing val-shard-000000.tar 0 0.0 GB 0
# writing val-shard-000001.tar 1000 0.1 GB 1000
🟢 

'\nIt took about 13 minutes to process 10.4K files (1.2 GB). If we keep 1K files per shard, \nprocessing 500K files could take around 11 hours. I’ll probably increase it to 10K \nfiles per shard, which would make each tar file around 1 GB and speed up the process.\n'

## 👉 **Dataset**  

**TODO:** Explain what dataset you are using for this project. Give a small overview of the classes, class distributions etc that can help anyone not familiar with the dataset get a better understanding of it. You can find more information about the data [here](https://registry.opendata.aws/amazon-bin-imagery/).  

In [10]:
from sagemaker.pytorch import PyTorch
# from sagemaker.inputs import TrainingInput
data_base_path = "s3://p5-amazon-bin-images/webdataset/"
# train_data = TrainingInput(data_base_path + "train/", 
#                            content_type="application/x-tar")
# val_data = TrainingInput(data_base_path + "val/", 
#                          content_type="application/x-tar")
# test_data = TrainingInput(data_base_path + "test/", 
#                           content_type="application/x-tar")
train_data_path = data_base_path + "train/train-shard-{000000..000007}.tar"
val_data_path = data_base_path + "val/val-shard-{000000..000001}.tar"
test_data_path = data_base_path + "test/test-shard-{000000..000001}.tar"
print(train_data_path)
## ⚠️ don't use prefix in output_path, cause source folder will be created 
## at bucket level, while other folders, e.g. debug-output, at prefix levle.
output_path = "s3://p5-amazon-bin-images-train/"  

## Manually set dataset sizes hyperparameters
l = 10441
split_ratio=[0.7, 0.15, 0.15]
train_data_size = int(l*split_ratio[0])
val_data_size = int(l*split_ratio[1])
test_data_size = l - train_data_size - val_data_size
print(f"train_size: {train_data_size}, val_size: {val_data_size}, test_size: {test_data_size}")

s3://p5-amazon-bin-images/webdataset/train/train-shard-{000000..000007}.tar
train_size: 7308, val_size: 1566, test_size: 1567


## 👉 **Model Training (Distributed Data Parallel)**  

**TODO:** This is the part where you can train a model. The type or architecture of the model you use is not important.   
**Note:** You will need to use the `train.py` script to train your model.

* Official document: [SageMaker distributed data parallel (SDP) with PyTorch](https://sagemaker-examples.readthedocs.io/en/latest/training/distributed_training/index.html#pytorch-distributed)   

* ⚠️ [Traning issues](https://gist.github.com/nov05/1bdc15eda0e781640b46ab28d38f45bd)   
* Training times
    * Train 2 epochs, val, test data sizes: 2K, 1K, 1K
        ```
        2025-02-06 22:26:36 Completed - Training job completed
        Training seconds: 874
        Billable seconds: 874
        CPU times: total: 14 s
        Wall time: 8min 45s
        ```

In [14]:
#TODO: Declare your model training hyperparameter.
#NOTE: You do not need to do hyperparameter tuning. You can use fixed hyperparameter values
from sagemaker.debugger import (
    Rule,
## debugger
    DebuggerHookConfig,
    rule_configs,
## profiler 
    ProfilerRule,
    ProfilerConfig,
    FrameworkProfile
)
## SageMaker will automatically append these as command-line arguments  
hyperparameters = {
    'epochs': 40,   
    'batch-size': 128,  ## ⚠️ 256 causes ml.g4dn.xlarge memory issue
    'opt-type': 'adamw',
    'opt-learning-rate': 1e-4,  
    'opt-weight-decay': 1e-5,  
    'lr-sched-step-size': 5,  
    'lr-sched-gamma': 0.5,
    'early-stopping-patience': 5,
    'model-arch': 'resnet34', 
    'wandb': True,  
    'debug': False, 
## input data 
    "train-data-path": train_data_path,
    "val-data-path": val_data_path,
    "test-data-path": test_data_path,
    "train-data-size": train_data_size, 
    "val-data-size": val_data_size,
    "test-data-size": test_data_size,
    "num-classes": 5,
    # "class-weights-dict": {
    #     1: 1.7004885993485341, 
    #     2: 0.9083079599826012, 
    #     3: 0.7832708177044261, 
    #     4: 0.8799831436999579, 
    #     5: 1.1137066666666666
    # },
}
rules = [
    Rule.sagemaker(rule_configs.poor_weight_initialization()),
    Rule.sagemaker(rule_configs.vanishing_gradient()),
    Rule.sagemaker(rule_configs.overfit()),
    Rule.sagemaker(rule_configs.overtraining()),
    Rule.sagemaker(rule_configs.loss_not_decreasing()),
    ProfilerRule.sagemaker(rule_configs.LowGPUUtilization()),
    ProfilerRule.sagemaker(rule_configs.ProfilerReport()),
]
hook_config = DebuggerHookConfig(
    hook_parameters={
        "train.save_interval": "100", 
        "eval.save_interval": "10"
    }
)
profiler_config = ProfilerConfig(
    system_monitor_interval_millis=500, 
    framework_profile_params=FrameworkProfile(num_steps=10)
)

[02/08/25 04:11:51] WARNING  Framework profiling will be deprecated from tensorflow 2.12 and     ]8;id=363763;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\deprecations.py\deprecations.py]8;;\:]8;id=673373;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\deprecations.py#34\34]8;;\
                             pytorch 2.0 in sagemaker>=2.                                                          
                             See: https://sagemaker.readthedocs.io/en/stable/v2.html for                           
                             details.                                                                              

In [15]:
%%time
#TODO: Create your training estimator
estimator = PyTorch(
    entry_point='train_v1.py',  # Your training script that defines the ResNet50 model and training loop
    source_dir='../scripts_train',  # Directory where your script and dependencies are stored
    role=sagemaker_role_arn,
    framework_version='1.13.1',  # Use the PyTorch version you need
    py_version='py39',
    instance_count=2,  ## multi-instance training, Udacity account level limit 2
    # instance_type='ml.p3.2xlarge',  ## 16GB, Use GPU instances for deep learning
    instance_type='ml.g4dn.xlarge',  ## 16GB, 1 GPU per instance
    output_path=output_path,  ## if not specify, output to the sagemaker default bucket
    hyperparameters=hyperparameters,
    # use_spot_instances=True,
## Debugger and profiler parameters
    # rules=rules,
    # debugger_hook_config=hook_config,    
    # profiler_config=profiler_config,
## Training using SMDataParallel Distributed Training Framework
    # distribution={"pytorchddp": {"enabled": True}}  # mpirun, activates SMDDP AllReduce OR AllGather
    # distribution={"torch_distributed": {"enabled": True}}  # torchrun, activates SMDDP AllGather
    distribution={"smdistributed": {"dataparallel": { "enabled": True}}},  # mpirun, activates SMDDP AllReduce OR AllGather
) 
# TODO: Fit your estimator
from datetime import datetime
estimator.fit(
    wait=True,  
    job_name=f"p5-amazon-bin-job-{datetime.now().strftime('%Y%m%d-%H%M%S')}", 
    ## Use WebDataset pipe to stream data instead 
    # inputs={
    #     "train": train_data,  
    #     "validation": val_data, 
    #     "test": test_data,
    # },  
)

[02/08/25 04:12:48] INFO     SageMaker Python SDK will collect telemetry to help us better  ]8;id=225654;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\telemetry\telemetry_logging.py\telemetry_logging.py]8;;\:]8;id=600298;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\telemetry\telemetry_logging.py#91\91]8;;\
                             understand our user's needs, diagnose issues, and deliver                             
                             additional features.                                                                  
                             To opt out of telemetry, please disable via TelemetryOptOut                           
                             parameter in SDK defaults config. For more information, refer                         
                             to                                                                                    
                             https://sagemaker.readthedocs.io/en/stable/overview.html#confi                        
                             guring-and-using-defaults-with-the-sagemaker-python-sdk.                              

[02/08/25 04:12:49] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=517720;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=883194;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py#679\679]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating training-job with name: p5-amazon-bin-job-20250208-041248     ]8;id=549938;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=348890;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#1042\1042]8;;\

2025-02-08 10:12:44 Starting - Starting the training job...
2025-02-08 10:12:59 Starting - Preparing the instances for training...
2025-02-08 10:13:39 Downloading - Downloading input data...
2025-02-08 10:14:04 Downloading - Downloading the training image...............
2025-02-08 10:17:06 Training - Training image download completed. Training in progress.bash: cannot set terminal process group (-1): Inappropriate ioctl for device
bash: no job control in this shell
/opt/conda/lib/python3.9/site-packages/paramiko/pkey.py:100: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "cipher": algorithms.TripleDES,
/opt/conda/lib/python3.9/site-packages/paramiko/transport.py:259: CryptographyDeprecationWarning: TripleDES has been moved to cryptography.hazmat.decrepit.ciphers.algorithms.TripleDES and will be removed from this module in 48.0.0.
  "class": algorithms.TripleDES,
2025

[02/08/25 05:06:50] WARNING  Job ended with status 'Stopped' rather than 'Completed'. This could    ]8;id=314512;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=748119;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#8666\8666]8;;\
                             mean the job timed out or stopped early for some other reason:                        
                             Consider checking whether it completed as you expect.                                 

Training seconds: 6296
Billable seconds: 6296
CPU times: total: 1min 30s
Wall time: 54min 4s


* Baseline accuracy 28.125%, [wandb logs](https://wandb.ai/nov05/udacity-awsmle-resnet34-amazon-bin/runs/p5-amazon-bin-job-20250207-044502-xfnie3-algo-2?nw=nwusernov05)  
  <img src="https://raw.githubusercontent.com/nov05/pictures/refs/heads/master/Udacity/20241119_aws-mle-nanodegree/2025-02-07%2005_28_14-baseline-p5-amazon-bin-job-20250207-044502-xfnie3-algo-2%20_%20udacity-awsmle-resnet.jpg" width=800>  

```log  
[1,mpirank:0,algo-1]<stdout>:👉 TEST: Average loss: 1.4330, Accuracy: 432/1536 (28.12%)  
[1,mpirank:0,algo-1]<stdout>:👉 Model saved at '/opt/ml/model/model.pth'
[1,mpirank:0,algo-1]<stderr>:wandb: Run history:
[1,mpirank:0,algo-1]<stderr>:wandb: Rank 0, train_loss ▇█▆▅▆▆▄█▅▅▅▅▅▄▃▃▄▅▄▃▆▅▄▄▄▃▄▄▅▄▃▃▆▆▄▅▅▁▃▃
[1,mpirank:0,algo-1]<stderr>:wandb: val_accuracy_epoch ▁▆▂█▄▃▃▆▃▂▅▆▆▇▅▄▅▇▇▆▇▆▆▆
[1,mpirank:0,algo-1]<stderr>:wandb:     val_loss_epoch █▆▇▅▅▅▄▆▄▅▂▂▂▂▂▁▂▂▁▂▁▂▁▁
[1,mpirank:0,algo-1]<stderr>:wandb: 
[1,mpirank:0,algo-1]<stderr>:wandb: Run summary:
[1,mpirank:0,algo-1]<stderr>:wandb: Rank 0, train_loss 1.3928
[1,mpirank:0,algo-1]<stderr>:wandb: val_accuracy_epoch 28.125
[1,mpirank:0,algo-1]<stderr>:wandb:     val_loss_epoch 1.43298
[1,mpirank:0,algo-1]<stderr>:wandb:
[1,mpirank:0,algo-1]<stderr>:wandb: 🚀 View run p5-amazon-bin-job-20250207-044502-xfnie3-algo-2 at: https://wandb.ai/nov05/udacity-awsmle-resnet34-amazon-bin/runs/p5-amazon-bin-job-20250207-044502-xfnie3-algo-2
[1,mpirank:0,algo-1]<stderr>:wandb: ⭐️ View project at: https://wandb.ai/nov05/udacity-awsmle-resnet34-amazon-bin
[1,mpirank:0,algo-1]<stderr>:wandb: Synced 6 W&B file(s), 0 media file(s), 0 artifact file(s) and 0 other file(s)
[1,mpirank:0,algo-1]<stderr>:wandb: Find logs at: ./wandb/run-20250207_104937-p5-amazon-bin-job-20250207-044502-xfnie3-algo-2/logs
```

* No class weights, 20 epochs, 36.46%  

```log 
[1,mpirank:0,algo-1]<stdout>:👉 VAL: Average loss: 1.4161, Accuracy: 560/1536 (36.46%)
[1,mpirank:0,algo-1]<stdout>:👉 Train Epoch: 20, Learning Rate: 1.25e-05
[1,mpirank:0,algo-1]<stdout>:👉 TEST: Average loss: 1.4599, Accuracy: 508/1536 (33.07%)
[1,mpirank:0,algo-1]<stdout>:🟢 Start saving the trained model...[1,mpirank:0,algo-1]<stdout>:
[1,mpirank:0,algo-1]<stdout>:👉 Model saved at '/opt/ml/model/model.pth'
[1,mpirank:0,algo-1]<stderr>:wandb: Run history:
[1,mpirank:0,algo-1]<stderr>:wandb: Rank 0, train_loss █▆▆█▆▅▇▄▅▅▆▅▄▄▄▃▃▄▄▄▃▄▃▄▆▆▅▄▄▃▃▄▃▄▃▄▃▁▄▄
[1,mpirank:0,algo-1]<stderr>:wandb: val_accuracy_epoch ▁▅▃▄▅▄▄▆▅▅▆▄▅▅▅▄▆▄██▇▆▆▇
[1,mpirank:0,algo-1]<stderr>:wandb:     val_loss_epoch █▃▄▃▂▂▃▂▂▄▂▃▂▂▂▂▁▂▁▁▂▂▂▂
[1,mpirank:0,algo-1]<stderr>:wandb: 
[1,mpirank:0,algo-1]<stderr>:wandb: Run summary:
[1,mpirank:0,algo-1]<stderr>:wandb: Rank 0, train_loss 1.45365
[1,mpirank:0,algo-1]<stderr>:wandb: val_accuracy_epoch 33.85417
[1,mpirank:0,algo-1]<stderr>:wandb:     val_loss_epoch 1.42693
[1,mpirank:0,algo-1]<stderr>:wandb: 
[1,mpirank:0,algo-1]<stderr>:wandb: 🚀 View run p5-amazon-bin-job-20250208-041248-0q3ao9-algo-1 at: https://wandb.ai/nov05/udacity-awsmle-resnet34-amazon-bin/runs/p5-amazon-bin-job-20250208-041248-0q3ao9-algo-1
[1,mpirank:0,algo-1]<stderr>:wandb: ⭐️ View project at: https://wandb.ai/nov05/udacity-awsmle-resnet34-amazon-bin
```

### Standout Suggestions
You do not need to perform the tasks below to finish your project. However, you can attempt these tasks to turn your project into a more advanced portfolio piece.

### Hyperparameter Tuning
**TODO:** Here you can perform hyperparameter tuning to increase the performance of your model. You are encouraged to 
- tune as many hyperparameters as you can to get the best performance from your model
- explain why you chose to tune those particular hyperparameters and the ranges.


In [ ]:
from sagemaker.pytorch import PyTorch
from datetime import datetime
data_base_path = "s3://p5-amazon-bin-images/webdataset/"
train_data_path = data_base_path + "train/train-shard-{000000..000007}.tar"
val_data_path = data_base_path + "val/val-shard-{000000..000001}.tar"
test_data_path = data_base_path + "test/test-shard-{000000..000001}.tar"
print(train_data_path)
output_path = "s3://p5-amazon-bin-images-train/"  
## Manually set dataset sizes hyperparameters
l = 10441  ## 10K dataset
split_ratio=[0.7, 0.15, 0.15]
train_data_size = int(l*split_ratio[0])
val_data_size = int(l*split_ratio[1])
test_data_size = l - train_data_size - val_data_size
print(f"train_size: {train_data_size}, val_size: {val_data_size}, test_size: {test_data_size}")
## s3://p5-amazon-bin-images/webdataset/train/train-shard-{000000..000007}.tar
## train_size: 7308, val_size: 1566, test_size: 1567
#TODO: Create your hyperparameter search space
## TODO: Declare your HP ranges, metrics etc.
from sagemaker.tuner import (
    IntegerParameter,
    CategoricalParameter,
    ContinuousParameter,
    HyperparameterTuner,
)
hyperparameters = {
    'epochs': 40,   
    # 'batch-size': 128,  ## ⚠️ 256 causes ml.g4dn.xlarge memory issue
    # 'opt-learning-rate': 8e-5,  
    # 'opt-weight-decay': 1e-5,  
    'lr-sched-step-size': 5,  
    'lr-sched-gamma': 0.5,
    'early-stopping-patience': 1000, ## use HPO Auto
    'model-arch': 'resnet34', 
    'wandb': True,  
    'debug': False, 
## input data 
    "train-data-path": train_data_path,
    "val-data-path": val_data_path,
    "test-data-path": test_data_path,
    "train-data-size": train_data_size, 
    "val-data-size": val_data_size,
    "test-data-size": test_data_size,
    "num-classes": 5,
    # "class-weights-dict": {
    #     1: 1.7004885993485341, 
    #     2: 0.9083079599826012, 
    #     3: 0.7832708177044261, 
    #     4: 0.8799831436999579, 
    #     5: 1.1137066666666666
    # },
}
hyperparameter_ranges = {
    # 'epochs': IntegerParameter(20, 40, scaling_type="Auto"),
    'batch-size': CategoricalParameter([64, 128]),
    'opt-learning-rate': ContinuousParameter(1e-6, 1e-5),
    'opt-weight-decay': ContinuousParameter(1e-5, 1e-4),
}
objective_metric_name = "eval_loss_epoch"
objective_type = "Minimize"
metric_definitions = [{
    "Name": "eval_loss_epoch", 
    "Regex": "👉 VAL: Average loss: ([0-9.]+)",
}]
## TODO: Create estimators for your HPs
estimator = PyTorch(
    entry_point='train_v1.py',  # Your training script that defines the ResNet50 model and training loop
    source_dir='../scripts_train',  # Directory where your script and dependencies are stored
    role=sagemaker_role_arn,
    framework_version='1.13.1',  # Use the PyTorch version you need
    py_version='py39',
    instance_count=2,  # 👈 Adjust based on the number of instances you want to use
    ## Running 2 ml.g4dn.xlarge instances concurrently would cost around $1.504 per hour in total.
    instance_type='ml.g4dn.xlarge',  ## 16GB $0.752/hr, Use GPU instances for deep learning
    # instance_type='ml.p3.2xlarge',  # 16GB $3.825/hr
    # instance_type='ml.p4d.24xlarge, ## 40*8GB $32.77/hr
    output_path=output_path,
    hyperparameters=hyperparameters,
    distribution={"smdistributed": {"dataparallel": { "enabled": True}}},  # 👈 mpirun, activates SMDDP AllReduce OR AllGather
    # use_spot_instances=True, ## By default, there is no quota for GPU spot instances available.
)
## TODO: Your HP tuner here
tuner = HyperparameterTuner(
    estimator,
    objective_metric_name,
    hyperparameter_ranges,
    metric_definitions,
    max_jobs=8,
    max_parallel_jobs=1,  ## this account limits gpu instance concurrent usage
    objective_type=objective_type,
    base_tuning_job_name='p5-amazon-bin-hpo',
    early_stopping_type='Auto',
)
tuner.fit(
    wait=False,  
) 
print("👉", tuner.latest_tuning_job.name)

s3://p5-amazon-bin-images/webdataset/train/train-shard-{000000..000007}.tar
train_size: 7308, val_size: 1566, test_size: 1567


[02/08/25 03:49:23] INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=388480;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=151784;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py#679\679]8;;\
                             instance_type, framework etc.                                                         

[02/08/25 03:49:24] WARNING  No finished training job found associated with this estimator.       ]8;id=404337;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\estimator.py\estimator.py]8;;\:]8;id=439146;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\estimator.py#1914\1914]8;;\
                             Please make sure this estimator is only used for building workflow                    
                             config                                                                                

                    INFO     image_uri is not presented, retrieving image_uri based on            ]8;id=652585;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py\image_uris.py]8;;\:]8;id=166413;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\image_uris.py#679\679]8;;\
                             instance_type, framework etc.                                                         

                    INFO     Creating hyperparameter tuning job with name:                          ]8;id=123128;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py\session.py]8;;\:]8;id=111499;file://d:\Users\guido\miniconda3\envs\awsmle_py310\lib\site-packages\sagemaker\session.py#3383\3383]8;;\
                             p5-amazon-bin-hpo-250208-0349                                                         

👉 p5-amazon-bin-hpo-250208-0349


In [ ]:
# TODO: Find the best hyperparameters
hpo_job_name = "p5-amazon-bin-hpo-250208-0349"

⚠️ Issue:  
> terminate called after throwing an instance of 'SMDDPConfigError'
  what():  The smdistributed.dataparallel functionality is invoked while the current job is not running as a distributed job. Please make sure you have the distribution strategy enabled in the SageMaker Estimator API according to https://docs.aws.amazon.com/sagemaker/latest/dg/data-parallel-use-api.html

### Model Profiling and Debugging
**TODO:** Use model debugging and profiling to better monitor and debug your model training job.

In [107]:
# TODO: Set up debugging and profiling rules and hooks

In [108]:
# TODO: Create and fit an estimator

In [109]:
# TODO: Plot a debugging output.

**TODO**: Is there some anomalous behaviour in your debugging output? If so, what is the error and how will you fix it?  
**TODO**: If not, suppose there was an error. What would that error look like and how would you have fixed it?

In [110]:
# TODO: Display the profiler output

### Model Deploying and Querying
**TODO:** Can you deploy your model to an endpoint and then query that endpoint to get a result?

In [111]:
# TODO: Deploy your model to an endpoint

In [112]:
# TODO: Run an prediction on the endpoint

In [113]:
# TODO: Remember to shutdown/delete your endpoint once your work is done

### Cheaper Training and Cost Analysis
**TODO:** Can you perform a cost analysis of your system and then use spot instances to lessen your model training cost?

In [114]:
# TODO: Cost Analysis

In [1]:
# TODO: Train your model using a spot instance
## By default, there is no quota for GPU spot instances available.